# Get output as midi files

In [14]:
# Standard imports
import numpy as np
import matplotlib.pyplot as plt
import os
import sys

# Music imports
from music21 import instrument, note, chord, stream
from PIL import Image, ImageOps
import mido

## From arrays to images

In [15]:
# Defining a function to use as key to sort images in music piece order
def get_alphabetical_numerical_sequences(x):
    """
    Returns number of a file within a given folder
    E.g.: returns 18 for file 'ballade2_instrument_0_18'
    """
    x = x.split('.')[0]
    parts = x.split('_')
    numerical = int((parts[-1]))
    return numerical

In [16]:
# Testing the function
get_alphabetical_numerical_sequences('ballade2_instrument_0_18')

18

In [4]:
# Creating a list of image names for one music piece (ballade2)
image_list = [image for image in os.listdir('../../data_image/ballade2/')]
image_list

FileNotFoundError: [Errno 2] No such file or directory: '../../data_image/ballade2/'

In [ ]:
# Sorting the images in order for one music piece (ballade2)
image_list = sorted(image_list, key = lambda x : get_alphabetical_numerical_sequences(x))
image_list

['ballade2_instrument_0_0.png',
 'ballade2_instrument_0_1.png',
 'ballade2_instrument_0_2.png',
 'ballade2_instrument_0_3.png',
 'ballade2_instrument_0_4.png',
 'ballade2_instrument_0_5.png',
 'ballade2_instrument_0_6.png',
 'ballade2_instrument_0_7.png',
 'ballade2_instrument_0_8.png',
 'ballade2_instrument_0_9.png',
 'ballade2_instrument_0_10.png',
 'ballade2_instrument_0_11.png',
 'ballade2_instrument_0_12.png',
 'ballade2_instrument_0_13.png',
 'ballade2_instrument_0_14.png',
 'ballade2_instrument_0_15.png',
 'ballade2_instrument_0_16.png',
 'ballade2_instrument_0_17.png',
 'ballade2_instrument_0_18.png']

In [5]:
# Transforming all images into arrays and adding them into a list
array_list = []
for image in image_list: 
    array = plt.imread(f"../../data_image/ballade2/{image}")
    array_list.append(array)
    
array_list

NameError: name 'image_list' is not defined

In [6]:
# Concatenating all arrays into one
array_conc = np.concatenate(array_list, axis = 1)
array_conc.shape

ValueError: need at least one array to concatenate

In [7]:
# Defining a function to create and save an array from an image
def array_to_image(array, output_file_path):
    """
    Save a numpy array of dimension 106 x 100 as a midi image into the corresponding subfolder
    'music_piece' of the 'data_output_image' folder
    """
    plt.imsave(output_file_path, array, cmap='gray')
    
# Question: should we add the transpose step here?

In [8]:
# Using the function for one music piece
array_to_image(array_conc,"../../data_output_image/ballade2/image_conc.png")

NameError: name 'array_conc' is not defined

## From images to midi

In [9]:
def column2notes(column, lowerBoundNote = 21):
    notes = []
    for i in range(len(column)):
        if column[i] > 255/2:
            notes.append(i+lowerBoundNote)
    return notes

In [10]:
def updateNotes(newNotes, prevNotes, resolution = 0.25): 
    res = {} 
    for note in newNotes:
        if note in prevNotes:
            res[note] = prevNotes[note] + resolution
        else:
            res[note] = resolution
    return res

In [11]:
def image2midi(image_path, lowerBoundNote = 21, resolution = 0.25):
    """
    From an existing image:
        - Convert to note
        - Save result as a midi file in the subfolder 'music_piece_name' of the 'data_output_sound' folder 
    """
    
    output_folder = f"../../data_output_midi/{image_path.split('/')[-2]}"
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    output_filename = os.path.join(output_folder, image_path.split("/")[-1].replace(".png",".mid"))
    print(output_filename)
    
    with ImageOps.grayscale(Image.open(image_path)) as image:
        im_arr = np.frombuffer(image.tobytes(), dtype=np.uint8)
        print(im_arr.shape)
        try:
            im_arr = im_arr.reshape((image.size[1], image.size[0]))
        except:
            im_arr = im_arr.reshape((image.size[1], image.size[0],3))
            im_arr = np.dot(im_arr, [0.33, 0.33, 0.33])
    
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model

    prev_notes = updateNotes(im_arr.T[0,:],{}, resolution = resolution)
    for column in im_arr.T[1:,:]:
        notes = column2notes(column, lowerBoundNote=lowerBoundNote)
        # pattern is a chord
        notes_in_chord = notes
        old_notes = prev_notes.keys()
        for old_note in old_notes:
            if not old_note in notes_in_chord:
                new_note = note.Note(old_note,quarterLength=prev_notes[old_note])
                new_note.storedInstrument = instrument.Piano()
                if offset - prev_notes[old_note] >= 0:
                    new_note.offset = offset - prev_notes[old_note]
                    output_notes.append(new_note)
                elif offset == 0:
                    new_note.offset = offset
                    output_notes.append(new_note)                    
                else:
                    print(offset,prev_notes[old_note],old_note)

        prev_notes = updateNotes(notes_in_chord,prev_notes)

        # increase offset each iteration so that notes do not stack
        offset += resolution

    for old_note in prev_notes.keys():
        new_note = note.Note(old_note,quarterLength=prev_notes[old_note])
        new_note.storedInstrument = instrument.Piano()
        new_note.offset = offset - prev_notes[old_note]

        output_notes.append(new_note)

    prev_notes = updateNotes(notes_in_chord,prev_notes)

    midi_stream = stream.Stream(output_notes)
    
    midi_stream.write('midi', fp=output_filename)

In [12]:
# Testing the function on concatenated array
image_path = "../../data_output_image/ballade2/image_conc.png"
image2midi(image_path)

../../data_output_midi/ballade2/image_conc.mid
(201400,)


## Investigating resolution impact on tempo

In [19]:
from imageio import imwrite
from music21 import converter, instrument, note, chord

In [26]:
#################################

def extractNote(element):
    return int(element.pitch.ps)

#################################

def extractDuration(element):
    return element.duration.quarterLength

#################################

def get_notes(notes_to_parse):

    """
    Get all the notes and chords from the midi files into a dictionary containing:
        - Start: unit time at which the note starts playing
        - Pitch: pitch of the note
        - Duration: number of time units the note is played for
    """
    durations = []
    notes = []
    start = []

    for element in notes_to_parse:
        if isinstance(element, note.Note):
            if element.isRest:
                continue

            start.append(element.offset)
            notes.append(extractNote(element))
            durations.append(extractDuration(element))

        elif isinstance(element, chord.Chord):
            if element.isRest:
                continue
            for chord_note in element:
                start.append(element.offset)
                durations.append(extractDuration(element))
                notes.append(extractNote(chord_note))

    return {"start":start, "pitch":notes, "dur":durations}

#################################

def midi2image(midi_path, max_repetitions = float("inf"), resolution = 0.25, lowerBoundNote = 21, upperBoundNote = 127, maxSongLength = 100):

    """
    1) Transform a midi file into a set of images:
        - Each image has a size of 106 (all notes between lowerBound and upperBound) x 100 time units (maxSongLength)
        - One time unit corresponds to 0.25 (resolution) beat from the original music
    2) Store images into the corresponding sub-folder (identified by music piece name) of the 'data_image' folder
    """

    output_folder = f"data_image/{midi_path.split('/')[-1].replace('.mid', '')}"
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    mid = converter.parse(midi_path)

    instruments = instrument.partitionByInstrument(mid)

    data = {}

    try:
        i=0
        for instrument_i in instruments.parts:
            notes_to_parse = instrument_i.recurse()

            notes_data = get_notes(notes_to_parse)
            if len(notes_data["start"]) == 0:
                continue

            if instrument_i.partName is None:
                data["instrument_{}".format(i)] = notes_data
                i+=1
            else:
                data[instrument_i.partName] = notes_data

    except:
        notes_to_parse = mid.flat.notes
        data["instrument_0"] = get_notes(notes_to_parse)

    for instrument_name, values in data.items():

        pitches = values["pitch"]
        durs = values["dur"]
        starts = values["start"]

        index = 0
        while index < max_repetitions:
            matrix = np.zeros((upperBoundNote-lowerBoundNote,maxSongLength))


            for dur, start, pitch in zip(durs, starts, pitches):
                dur = int(dur/resolution)
                start = int(start/resolution)

                if not start > index*(maxSongLength+1) or not dur+start < index*maxSongLength:
                    for j in range(start,start+dur):
                        if j - index*maxSongLength >= 0 and j - index*maxSongLength < maxSongLength:
                            matrix[pitch-lowerBoundNote,j - index*maxSongLength] = 255

            # if matrix.any(): # If matrix contains no notes (only zeros) don't save it
            output_filename = os.path.join(output_folder, midi_path.split('/')[-1].replace(".mid",f"_{instrument_name}_{index}.png"))
            imwrite(output_filename,matrix.astype(np.uint8))
            index += 1
            # else:
                # break

#################################

In [27]:
midi_path = '../../data_raw/ballade2.mid'
midi2image(midi_path)

KeyboardInterrupt: 

In [23]:
midi_path.split('/')

['..', '..', 'data_raw', 'ballade2.mid']